In [ ]:
%%bash
pip install --no-cache-dir -qU pip wheel
pip install --no-cache-dir -qU numpy pandas matplotlib seaborn scikit-learn
pip install --no-cache-dir -qU tensorflow-gpu
pip check

# Saving model architecture only

In this reading you will learn how to save a model's architecture, but not its weights.

In [2]:
import os
import numpy as np
import pandas as pd
import json

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')
sns.set(font='DejaVu Sans')

import tensorflow as tf
tf.keras.utils.set_random_seed(42)
tf.get_logger().setLevel('ERROR')

In previous videos and notebooks you have learned how to save a model's weights, as well as the entire model - weights and architecture.

### Accessing a model's configuration

A model's *configuration* refers to its architecture. TensorFlow has a convenient way to retrieve a model's architecture as a dictionary. We start by creating a simple fully connected feedforward neural network with 1 hidden layer.

In [4]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', 
        input_shape=(32, 32, 3), name='dense_1'),
    tf.keras.layers.Dense(units=10, activation='softmax', 
        name='dense_2')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 32, 32, 32)        128       
                                                                 
 dense_2 (Dense)             (None, 32, 32, 10)        330       
                                                                 
Total params: 458
Trainable params: 458
Non-trainable params: 0
_________________________________________________________________


A TensorFlow model has an inbuilt method `get_config` which returns the model's architecture as a dictionary:

In [5]:
# Get the model config
config_dict = model.get_config()
config_dict

{'name': 'sequential_1',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32, 32, 3),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_1_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 32, 32, 3),
    'units': 32,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'softmax',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': N

### Creating a new model from the config

A new TensorFlow model can be created from this config dictionary. This model will have reinitialized weights, which are not the same as the original model.

In [6]:
# Create a model from the config dictionary
model_same_config = tf.keras.Sequential.from_config(config_dict)
model_same_config.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 32, 32, 32)        128       
                                                                 
 dense_2 (Dense)             (None, 32, 32, 10)        330       
                                                                 
Total params: 458
Trainable params: 458
Non-trainable params: 0
_________________________________________________________________


We can check explicitly that the config of both models is the same, but the weights are not: 

In [7]:
# Check the new model is the same architecture
print('Same config:', model.get_config()['layers'] == \
    model_same_config.get_config()['layers'])
print('Same value for first weight matrix:', np.allclose(
    model.weights[0].numpy(), model_same_config.weights[0].numpy()))

Same config: True
Same value for first weight matrix: False


For models that are not `Sequential` models, use `tf.keras.Model.from_config` instead of `tf.keras.Sequential.from_config`.

### Other file formats: JSON and YAML
It is also possible to obtain a model's config in JSON or YAML formats. This follows the same pattern:

In [8]:
# Convert the model to JSON
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32, 32, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_1_input"}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "batch_input_shape": [null, 32, 32, 3], "units": 32, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_re

The JSON format can easily be written out and saved as a file:

In [9]:
# Write out JSON config file
with open('config.json', 'w') as f:
    json.dump(json_string, f)
del json_string

In [10]:
# Read in JSON config file again
with open('config.json', 'r') as f:
    json_string = json.load(f)

In [11]:
# Reinitialize the model
model_same_config = tf.keras.models.model_from_json(json_string)
model_same_config.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 32, 32, 32)        128       
                                                                 
 dense_2 (Dense)             (None, 32, 32, 10)        330       
                                                                 
Total params: 458
Trainable params: 458
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Check the new model is the same architecture, but different weights
print('Same config:', model.get_config()['layers'] == \
    model_same_config.get_config()['layers'])
print('Same value for first weight matrix:', np.allclose(
    model.weights[0].numpy(), model_same_config.weights[0].numpy()))

Same config: True
Same value for first weight matrix: False


In [13]:
# Clear file
! rm config.json

### Further reading and resources
* https://www.tensorflow.org/guide/keras/save_and_serialize#architecture-only_saving
* https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model